In [1]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import os
import io
from io import StringIO
from datetime import datetime as dt
import numpy as np

In [2]:
#path to csv file
netflix_file = "../ETL_Project/netflix_titles.csv"

#Read in csv file  
netflix_df = pd.read_csv(netflix_file)

#split rows that have multiple countries in a single row and stack them into the a new df
netflix_df_2 = netflix_df.country.str.split(',').apply(pd.Series, 1).stack()

#drop the index from intial df and join both dfs
netflix_df_2.index = netflix_df_2.index.droplevel(-1)
netflix_df_2.name = 'country'
del netflix_df['country']
netflix_df = netflix_df.join(netflix_df_2)

In [3]:
#removed unnecessary columns
netflix_col=["title", "type", "date_added", "country", "rating"]
netflix_transformed=netflix_df[netflix_col].copy()

#count of values in the columns
netflix_transformed.count()


title         6234
type          6234
date_added    6223
country       5758
rating        6224
dtype: int64

In [4]:
#drop rows that do not have a value in 'country' column
netflix_transformed= netflix_transformed.dropna(subset=['country'])
netflix_transformed.count()

title         5758
type          5758
date_added    5749
country       5758
rating        5752
dtype: int64

In [5]:
#drop rows that do not have a value in 'date added' column
netflix_transformed= netflix_transformed.dropna(subset=['date_added'])
print(netflix_transformed.count())

#view the df
netflix_transformed

title         5749
type          5749
date_added    5749
country       5749
rating        5743
dtype: int64


,title,type,date_added,country,rating
0,Norm of the North: King Sized Adventure,Movie,9-Sep-19,United States,TV-PG
1,Jandino: Whatever it Takes,Movie,9-Sep-16,United Kingdom,TV-MA
2,Transformers Prime,TV Show,8-Sep-18,United States,TV-Y7-FV
3,Transformers: Robots in Disguise,TV Show,8-Sep-18,United States,TV-Y7
4,#realityhigh,Movie,8-Sep-17,United States,TV-14
...,...,...,...,...,...
6218,Talking Tom and Friends,TV Show,"April 10, 2019",Cyprus,TV-G
6219,Pokémon the Series,TV Show,"April 1, 2019",Japan,TV-Y7-FV
6220,Justin Time,TV Show,"April 1, 2016",Canada,TV-Y
6221,Terrace House: Boys & Girls in the City,TV Show,"April 1, 2016",Japan,TV-14


In [6]:
#extract rows that have "United States" in the 'country' column
netflix_transformed_US = netflix_transformed.loc[(netflix_transformed["country"]=="United States")]
netflix_transformed_US.count()



title         2296
type          2296
date_added    2296
country       2296
rating        2293
dtype: int64

In [7]:
#extract rows that have "Canada" in the 'country' column
netflix_transformed_CA = netflix_transformed.loc[(netflix_transformed["country"]== "Canada")]
netflix_transformed_CA.count()

title         206
type          206
date_added    206
country       206
rating        206
dtype: int64

In [8]:
#######Method 3 #######
#join the United States and Canada dfs
netflix_transformed = netflix_transformed_US.append(netflix_transformed_CA)
print(netflix_transformed)
print(netflix_transformed.count())

                                        title     type       date_added  \
0     Norm of the North: King Sized Adventure    Movie         9-Sep-19   
2                          Transformers Prime  TV Show         8-Sep-18   
3            Transformers: Robots in Disguise  TV Show         8-Sep-18   
4                                #realityhigh    Movie         8-Sep-17   
8                                Fire Chasers  TV Show         8-Sep-17   
...                                       ...      ...              ...   
6188             Roman Empire: Reign of Blood  TV Show    April 5, 2019   
6192                        Kim's Convenience  TV Show    April 3, 2019   
6208                                Lost Girl  TV Show   April 17, 2016   
6209                                Mr. Young  TV Show   April 16, 2019   
6220                              Justin Time  TV Show    April 1, 2016   

            country    rating  
0     United States     TV-PG  
2     United States  TV-Y7-FV  
3  

In [9]:
# drop duplicated indexes
netflix_transformed.drop_duplicates(keep='first')

#view the df
print(netflix_transformed)

                                        title     type       date_added  \
0     Norm of the North: King Sized Adventure    Movie         9-Sep-19   
2                          Transformers Prime  TV Show         8-Sep-18   
3            Transformers: Robots in Disguise  TV Show         8-Sep-18   
4                                #realityhigh    Movie         8-Sep-17   
8                                Fire Chasers  TV Show         8-Sep-17   
...                                       ...      ...              ...   
6188             Roman Empire: Reign of Blood  TV Show    April 5, 2019   
6192                        Kim's Convenience  TV Show    April 3, 2019   
6208                                Lost Girl  TV Show   April 17, 2016   
6209                                Mr. Young  TV Show   April 16, 2019   
6220                              Justin Time  TV Show    April 1, 2016   

            country    rating  
0     United States     TV-PG  
2     United States  TV-Y7-FV  
3  

In [10]:
# check for duplicated in Movie titles
netflix_transformed_dups = netflix_transformed.duplicated(subset=['title'])
netflix_transformed_dups

0       False
2       False
3       False
4       False
8       False
        ...  
6188    False
6192    False
6208    False
6209    False
6220    False
Length: 2502, dtype: bool

In [11]:
#count of duplicates found
netflix_transformed_dups.loc[(netflix_transformed_dups == True)].count()

9

In [12]:
# Drop rows with Duplicate Movie title
cleaned_netflix_transformed = netflix_transformed.drop_duplicates(subset=["title"], keep='last')
cleaned_netflix_transformed

,title,type,date_added,country,rating
0,Norm of the North: King Sized Adventure,Movie,9-Sep-19,United States,TV-PG
2,Transformers Prime,TV Show,8-Sep-18,United States,TV-Y7-FV
3,Transformers: Robots in Disguise,TV Show,8-Sep-18,United States,TV-Y7
4,#realityhigh,Movie,8-Sep-17,United States,TV-14
8,Fire Chasers,TV Show,8-Sep-17,United States,TV-MA
...,...,...,...,...,...
6188,Roman Empire: Reign of Blood,TV Show,"April 5, 2019",Canada,TV-MA
6192,Kim's Convenience,TV Show,"April 3, 2019",Canada,TV-14
6208,Lost Girl,TV Show,"April 17, 2016",Canada,TV-14
6209,Mr. Young,TV Show,"April 16, 2019",Canada,TV-G


In [13]:
sub_file = "../ETL_Project/DataNetflixSubscriber.csv"
sub_df = pd.read_csv(sub_file)
sub_df.head()

,Area,Years,Subscribers
0,United States and Canada,Q1 - 2018,60909000
1,"Europe, Middle East and Africa",Q1 - 2018,29339000
2,Latin America,Q1 - 2018,21260000
3,Asia-Pacific,Q1 - 2018,7394000
4,United States and Canada,Q2 - 2018,61870000


In [14]:
cleaned_netflix_transformed = cleaned_netflix_transformed.rename(columns={"title": "Title","type": "Category",\
                                                                          "date_added": "Years","country": "Country",\
                                                                          "rating": "Rating"})

print(cleaned_netflix_transformed)

                                        Title Category            Years  \
0     Norm of the North: King Sized Adventure    Movie         9-Sep-19   
2                          Transformers Prime  TV Show         8-Sep-18   
3            Transformers: Robots in Disguise  TV Show         8-Sep-18   
4                                #realityhigh    Movie         8-Sep-17   
8                                Fire Chasers  TV Show         8-Sep-17   
...                                       ...      ...              ...   
6188             Roman Empire: Reign of Blood  TV Show    April 5, 2019   
6192                        Kim's Convenience  TV Show    April 3, 2019   
6208                                Lost Girl  TV Show   April 17, 2016   
6209                                Mr. Young  TV Show   April 16, 2019   
6220                              Justin Time  TV Show    April 1, 2016   

            Country    Rating  
0     United States     TV-PG  
2     United States  TV-Y7-FV  
3  

In [15]:
# convert the 'Years' column to datetime format
cleaned_netflix_transformed["Years"]=pd.to_datetime(cleaned_netflix_transformed['Years'])

#created a start and end date to grab only the rows between 2018-2020
start_date = '12-31-2017'
end_date = '12-31-2020'

#created a variable to hold the dates
year_filtered_df = (cleaned_netflix_transformed['Years'] > start_date) & (cleaned_netflix_transformed['Years'] <= end_date)

#filtered the rows using the dates variable created
cleaned_netflix_transformed = cleaned_netflix_transformed.loc[year_filtered_df]
cleaned_netflix_transformed

,Title,Category,Years,Country,Rating
0,Norm of the North: King Sized Adventure,Movie,2019-09-09,United States,TV-PG
2,Transformers Prime,TV Show,2018-09-08,United States,TV-Y7-FV
3,Transformers: Robots in Disguise,TV Show,2018-09-08,United States,TV-Y7
27,City of Joy,Movie,2018-09-07,United States,TV-MA
31,Sierra Burgess Is A Loser,Movie,2018-09-07,United States,PG-13
...,...,...,...,...,...
6145,Workin' Moms,TV Show,2019-08-29,Canada,TV-MA
6174,Heartland,TV Show,2019-08-01,Canada,TV-PG
6188,Roman Empire: Reign of Blood,TV Show,2019-04-05,Canada,TV-MA
6192,Kim's Convenience,TV Show,2019-04-03,Canada,TV-14


In [16]:
cleaned_netflix_transformed["Years"]=pd.to_datetime(cleaned_netflix_transformed['Years'])

#Using Years
cleaned_netflix_transformed["Years"]=cleaned_netflix_transformed["Years"].dt.strftime("%Y")

cleaned_netflix_transformed.head()

,Title,Category,Years,Country,Rating
0,Norm of the North: King Sized Adventure,Movie,2019,United States,TV-PG
2,Transformers Prime,TV Show,2018,United States,TV-Y7-FV
3,Transformers: Robots in Disguise,TV Show,2018,United States,TV-Y7
27,City of Joy,Movie,2018,United States,TV-MA
31,Sierra Burgess Is A Loser,Movie,2018,United States,PG-13


In [17]:
cleaned_netflix_transformed=cleaned_netflix_transformed.drop(columns=['Title'])
cleaned_netflix_transformed

,Category,Years,Country,Rating
0,Movie,2019,United States,TV-PG
2,TV Show,2018,United States,TV-Y7-FV
3,TV Show,2018,United States,TV-Y7
27,Movie,2018,United States,TV-MA
31,Movie,2018,United States,PG-13
...,...,...,...,...
6145,TV Show,2019,Canada,TV-MA
6174,TV Show,2019,Canada,TV-PG
6188,TV Show,2019,Canada,TV-MA
6192,TV Show,2019,Canada,TV-14


In [18]:
#filtered dataframe from specific columns
sub_col=["Area","Years","Subscribers"]
sub_transformed= sub_df[sub_col].copy()
year_sub=sub_transformed["Years"].str.split(" - ").str[1]
sub_transformed["Years"]=sub_transformed["Years"].str.split(" - ").str[1]

In [19]:

print(sub_transformed)
sub_transformed.info()

                               Area Years  Subscribers
0          United States and Canada  2018     60909000
1   Europe,  Middle East and Africa  2018     29339000
2                     Latin America  2018     21260000
3                      Asia-Pacific  2018      7394000
4          United States and Canada  2018     61870000
5   Europe,  Middle East and Africa  2018     31317000
6                     Latin America  2018     22795000
7                      Asia-Pacific  2018      8372000
8          United States and Canada  2018     63010000
9   Europe,  Middle East and Africa  2018     33836000
10                    Latin America  2018     24115000
11                     Asia-Pacific  2018      9461000
12         United States and Canada  2018     64757000
13  Europe,  Middle East and Africa  2018     37818000
14                    Latin America  2018     26077000
15                     Asia-Pacific  2018     10607000
16         United States and Canada  2019     66633000
17  Europe

In [20]:
#filtering using just United States and Canada

cleaned_sub_transformed=sub_transformed.loc[(sub_transformed["Area"]=="United States and Canada")]

print(cleaned_sub_transformed)
cleaned_sub_transformed.info()

                        Area Years  Subscribers
0   United States and Canada  2018     60909000
4   United States and Canada  2018     61870000
8   United States and Canada  2018     63010000
12  United States and Canada  2018     64757000
16  United States and Canada  2019     66633000
20  United States and Canada  2019     66501000
24  United States and Canada  2019     67114000
28  United States and Canada  2019     67662000
32  United States and Canada  2020     69969000
36  United States and Canada  2020     72904000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 36
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Area         10 non-null     object
 1   Years        10 non-null     object
 2   Subscribers  10 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 320.0+ bytes


In [38]:
#database connection
connection_string ="postgres:Shjais2014@localhost:5432/Netflix"
engine = create_engine(f'postgresql://{connection_string}')

In [39]:
engine.table_names()

['netflix', 'subscribers']

In [40]:
### Load DataFrames into database
cleaned_netflix_transformed.to_sql(name='netflix', con=engine, if_exists='append')
cleaned_sub_transformed.to_sql(name='subscribers', con=engine, if_exists='append')

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "index" of relation "netflix" does not exist
LINE 1: INSERT INTO netflix (index, "Category", "Years", "Country", ...
                             ^

[SQL: INSERT INTO netflix (index, "Category", "Years", "Country", "Rating") VALUES (%(index)s, %(Category)s, %(Years)s, %(Country)s, %(Rating)s)]
[parameters: ({'index': 0, 'Category': 'Movie', 'Years': '2019', 'Country': 'United States', 'Rating': 'TV-PG'}, {'index': 2, 'Category': 'TV Show', 'Years': '2018', 'Country': 'United States', 'Rating': 'TV-Y7-FV'}, {'index': 3, 'Category': 'TV Show', 'Years': '2018', 'Country': 'United States', 'Rating': 'TV-Y7'}, {'index': 27, 'Category': 'Movie', 'Years': '2018', 'Country': 'United States', 'Rating': 'TV-MA'}, {'index': 31, 'Category': 'Movie', 'Years': '2018', 'Country': 'United States', 'Rating': 'PG-13'}, {'index': 40, 'Category': 'Movie', 'Years': '2018', 'Country': 'United States', 'Rating': 'R'}, {'index': 45, 'Category': 'Movie

In [31]:
#Query confirmed through SQL database for neflix 
pd.read_sql_query('select * from netflix', con=engine).head()

,Category,Years,Country,Rating
0,Movie,2019,United States,TV-PG
1,TV Show,2018,United States,TV-Y7-FV
2,TV Show,2018,United States,TV-Y7
3,Movie,2018,United States,TV-MA
4,Movie,2018,United States,PG-13


In [32]:
#Query confirmed through SQL database for Netflix subscribers

pd.read_sql_query('select * from subscribers', con=engine).head()

,Area,Years,Subscribers
0,United States and Canada,2018,60909000
1,United States and Canada,2018,61870000
2,United States and Canada,2018,63010000
3,United States and Canada,2018,64757000
4,United States and Canada,2019,66633000


In [ ]:
#The numbers of subscribers 
num_sub =sub_transformed.groupby("Years")
num_sub

In [ ]:
max_sub = num_sub["Subscribers"].max()
max_sub

In [ ]:
sub_df= pd.DataFrame({"Subscriber Count": max_sub,})
sub_df.head()

In [ ]:
grouped_dm = sub_df.groupby(["Years"]).count()["Subscriber Count"]
grouped_dm

mice_chart= grouped_dm.plot(kind="bar", title= "What year has the most subsribers")
mice_chart.set_xlabel("Years")
mice_chart.set_ylabel("Subscriber Count")
#plt.xlim(-0.10, len(x_axis)-.5)
#plt.ylim(0, max(sub_df["Subscriber Count"])+10)
plt.show()
plt.tight_layout()

In [ ]:
gender_count = sub_df.groupby('Years')
gender_mouse_count = pd.DataFrame(gender_count['Subscriber Count'].count())
gender_mouse_count.head()
gender_mouse_count.plot(kind="pie", y='Subscriber Count', autopct="%1.1f%%", colors = ["lightpink", "lightblue"], shadow=True, legend=False)
plt.axis("equal")
plt.title("Male v. Female Percent")
plt.show()